In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

Building prefix dict from the default dictionary ...


DEBUG:jieba:Building prefix dict from the default dictionary ...


Loading model from cache C:\Users\xinre\AppData\Local\Temp\jieba.cache


DEBUG:jieba:Loading model from cache C:\Users\xinre\AppData\Local\Temp\jieba.cache


Loading model cost 0.488 seconds.


DEBUG:jieba:Loading model cost 0.488 seconds.


Prefix dict has been built successfully.


DEBUG:jieba:Prefix dict has been built successfully.


## Joi

In [2]:
hps = utils.get_hparams_from_file("./configs/joi.json")

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

In [5]:
import numpy as np

_ = utils.load_checkpoint(utils.latest_checkpoint_path("logs/joi/"), net_g, None)

stn_tst = get_text("这是一首简单的小情歌", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.8, noise_scale_w=1.0, length_scale=1.0)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))


logs/joi\G_4000.pth
INFO:root:Loaded checkpoint 'logs/joi\G_4000.pth' (iteration 572)
